In [1]:
from pot_correction import *
from uncertain_dp2 import *
from itertools import islice

In [2]:
DNA_LENGTH = 8000          
READ_lENGTH = 50 
READ_COUNT = 8000
KMER_LENGTH = 18
PROBABILISTIC_ALIGNMENT_BONUS = 2

In [3]:
dna = generate_dna(DNA_LENGTH)
reads = [generate_read(dna, READ_lENGTH, lambda base: gauss_unsharp_uncertainty_generator(base, 0.46)) for _ in range(READ_COUNT)]

In [4]:
most_likely_restorer_error_rate(reads)

[0.86, 0.94, 0.86, 0.88, 0.84, 0.82, 0.86, 0.86, 0.88, 0.84, 0.82, 0.9, 0.96, 0.84, 0.9, 0.92, 0.86, 0.88, 0.84, 0.72, 0.8, 0.8, 0.76, 0.86, 0.88, 0.88, 0.86, 0.88, 0.88, 0.94, 0.76, 0.88, 0.9, 0.76, 0.84, 0.88, 0.86, 0.9, 0.9, 0.96, 0.8, 0.88, 0.86, 0.72, 0.86, 0.72, 0.86, 0.88, 0.8, 0.88, 0.8, 0.9, 0.8, 0.84, 0.9, 0.84, 0.86, 0.84, 0.84, 0.9, 0.88, 0.86, 0.8, 0.8, 0.82, 0.84, 0.9, 0.92, 0.96, 0.88, 0.86, 0.84, 0.88, 0.82, 0.76, 0.8, 0.8, 0.88, 0.88, 0.88, 0.9, 0.84, 0.92, 0.86, 0.84, 0.88, 0.9, 0.84, 0.84, 0.8, 0.88, 0.8, 0.9, 0.9, 0.82, 0.86, 0.78, 0.84, 0.86, 0.92, 0.84, 0.86, 0.78, 0.86, 0.68, 0.9, 0.86, 0.78, 0.82, 0.86, 0.82, 0.84, 0.72, 0.86, 0.82, 0.78, 0.8, 0.88, 0.84, 0.94, 0.94, 0.74, 0.86, 0.84, 0.82, 0.86, 0.88, 0.86, 0.84, 0.86, 0.84, 0.78, 0.86, 0.8, 0.84, 0.86, 0.84, 0.9, 0.86, 0.9, 0.86, 0.86, 0.86, 0.86, 0.8, 0.86, 0.84, 0.78, 0.92, 0.86, 0.86, 0.84, 0.88, 0.88, 0.92, 0.86, 0.92, 0.8, 0.78, 0.82, 0.9, 0.78, 0.84, 0.82, 0.92, 0.8, 0.88, 0.84, 0.86, 0.88, 0.9, 0.82, 0.

0.14871000000000167

# Finding similar reads with a kmer-hash

In [5]:
kmer_hash = KmerHash(KMER_LENGTH, reads)

In [6]:
r = reads[6]
kmer = most_likely_restorer(r.uncertain_text)[:KMER_LENGTH]
adjacent_reads = list(set(kmer_hash.get_adjacent_reads(kmer, 4)))

In [7]:
#kmer = most_likely_restorer(r.uncertain_text)[20:20+KMER_LENGTH]
#print(kmer)
#adjacent_reads2 = list(set(kmer_hash.get_adjacent_reads(kmer, 3)))

In [8]:
len(adjacent_reads)

5

In [9]:
#adjacent_reads = list(set(adjacent_reads).intersection(set(adjacent_reads2)))
#adjacent_reads

In [10]:
adjacent_reads = adjacent_reads[:10]

In [11]:
[r.start_position for r in adjacent_reads]

[3269, 3290, 3289, 3260, 3261]

In [12]:
most_likely_restorer_error_rate(adjacent_reads)

[0.88, 0.86, 0.82, 0.94, 0.78]


0.1439999999999999

In [17]:
adjacent_reads

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

# Partial Order Graph Correction

In [13]:

probabilistic_corrected_reads = correct_reads_with_consens(adjacent_reads,probabilistic=True, alignment_bonus=PROBABILISTIC_ALIGNMENT_BONUS)

In [14]:
probabilistic_corrected_reads

[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [18]:
most_likely_restorer_error_rate(probabilistic_corrected_reads)

[0.98, 0.88, 0.94, 0.98, 0.96]


0.051999999999999935

# Pot correction

In [15]:
pot = Pot()
pot.members = set(adjacent_reads)
in_pot_alignment, in_pot_score = get_in_pot_alignment(pot)
restored_text = most_likely_pot_string(in_pot_alignment)
corrected_reads = correct_reads_with_restored_text(in_pot_alignment, restored_text)

In [16]:
most_likely_restorer_error_rate(corrected_reads)

[0.98, 0.94, 0.98, 0.94, 0.96]


0.040000000000000036